### 1.Entity Rules

In [1]:
import spacy

In [2]:
nlp = spacy.load("en_core_web_sm")
text = "West Chestertenfieldville was referenced in Mr. Deeds."

In [3]:
doc = nlp(text)

In [4]:
for ent in doc.ents:
    print(ent.text, ent.label_)

West Chestertenfieldville GPE
Deeds PERSON


In [5]:
ruler = nlp.add_pipe("entity_ruler")

In [6]:
nlp.analyze_pipes()

{'summary': {'tok2vec': {'assigns': ['doc.tensor'],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'tagger': {'assigns': ['token.tag'],
   'requires': [],
   'scores': ['tag_acc'],
   'retokenizes': False},
  'parser': {'assigns': ['token.dep',
    'token.head',
    'token.is_sent_start',
    'doc.sents'],
   'requires': [],
   'scores': ['dep_uas',
    'dep_las',
    'dep_las_per_type',
    'sents_p',
    'sents_r',
    'sents_f'],
   'retokenizes': False},
  'attribute_ruler': {'assigns': [],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'lemmatizer': {'assigns': ['token.lemma'],
   'requires': [],
   'scores': ['lemma_acc'],
   'retokenizes': False},
  'ner': {'assigns': ['doc.ents', 'token.ent_iob', 'token.ent_type'],
   'requires': [],
   'scores': ['ents_f', 'ents_p', 'ents_r', 'ents_per_type'],
   'retokenizes': False},
  'entity_ruler': {'assigns': ['doc.ents', 'token.ent_type', 'token.ent_iob'],
   'requires': [],
   'scores': ['ents_f', 'ent

In [29]:
patterns = [
     {"label": "GPE", "pattern": "West Chestertenfieldville"}
]

In [30]:
ruler.add_patterns(patterns)

In [31]:
doc2 = nlp(text)
for ent in doc2.ents:
    print(ent.text, ent.label_ )

West Chestertenfieldville GPE
Deeds PERSON


In [32]:
nlp2 = spacy.load("en_core_web_sm")

In [33]:
ruler = nlp2.add_pipe("entity_ruler", before = "ner")

In [34]:
ruler.add_patterns(patterns)

In [35]:
doc = nlp2(text)

In [36]:
for ent in doc.ents:
    print(ent.text, ent.label_)

West Chestertenfieldville GPE
Deeds PERSON


In [43]:
nlp3 = spacy.load("en_core_web_sm")

In [44]:
ruler = nlp3.add_pipe("entity_ruler", before="ner")

In [45]:
patterns = [
    {"label": "GPE", "pattern": "West Chestertenfielville"},
    {"label": "FILM", "pattern": "Mr. Deeds"}
]

In [46]:
ruler.add_patterns(patterns)

In [48]:
doc = nlp3(text)

In [49]:
for ent in doc.ents:
    print(ent.text, ent.label_)

West Chestertenfieldville GPE
Mr. Deeds FILM


In [ ]:
#toponym resolution


### 2. Spacy Matcher

In [50]:
import spacy

In [51]:
from spacy.matcher import Matcher

In [52]:
nlp = spacy.load("en_core_web_sm")

In [54]:
matcher = Matcher(nlp.vocab)

In [55]:
pattern = [{"LIKE_EMAIL": True}]
matcher.add("EMAIL_ADDRESS", [pattern])

In [56]:
doc = nlp("this is an email address: wmattingly@aol.com")
matches = matcher(doc)

In [57]:
print(matches) #Lexeme , start tken, end token.

[(16571425990740197027, 6, 7)]


In [58]:
print(nlp.vocab[matches[0][0]].text)

EMAIL_ADDRESS


In [59]:
with open("data/wiki_mlk.txt", "r") as f :
    text = f.read()

In [60]:
print(text)

Martin Luther King Jr. (born Michael King Jr.; January 15, 1929 â€“ April 4, 1968) was an American Baptist minister and activist who became the most visible spokesman and leader in the American civil rights movement from 1955 until his assassination in 1968. King advanced civil rights through nonviolence and civil disobedience, inspired by his Christian beliefs and the nonviolent activism of Mahatma Gandhi. He was the son of early civil rights activist and minister Martin Luther King Sr.

King participated in and led marches for blacks' right to vote, desegregation, labor rights, and other basic civil rights.[1] King led the 1955 Montgomery bus boycott and later became the first president of the Southern Christian Leadership Conference (SCLC). As president of the SCLC, he led the unsuccessful Albany Movement in Albany, Georgia, and helped organize some of the nonviolent 1963 protests in Birmingham, Alabama. King helped organize the 1963 March on Washington, where he delivered his famou

In [61]:
nlp = spacy.load("en_core_web_sm")

In [62]:
matcher = Matcher(nlp.vocab)
pattern = [{"POS": "PROPN"}]
matcher.add("PROPER_NOUN", [pattern])
doc = nlp(text)
matches = matcher(doc)
print(len(matches))
for match in matches[:10]:
    print(match, doc[match[1]:match[2]])

99
(451313080118390996, 0, 1) Martin
(451313080118390996, 1, 2) Luther
(451313080118390996, 2, 3) King
(451313080118390996, 3, 4) Jr.
(451313080118390996, 6, 7) Michael
(451313080118390996, 7, 8) King
(451313080118390996, 8, 9) Jr.
(451313080118390996, 10, 11) January
(451313080118390996, 16, 17) April
(451313080118390996, 24, 25) Baptist


In [63]:
matcher = Matcher(nlp.vocab)
pattern = [{"POS": "PROPN", "OP": "+"}]
matcher.add("PROPER_NOUN", [pattern])
doc = nlp(text)
matches = matcher(doc)
print(len(matches))
for match in matches[:10]:
    print(match, doc[match[1]:match[2]])

168
(451313080118390996, 0, 1) Martin
(451313080118390996, 0, 2) Martin Luther
(451313080118390996, 1, 2) Luther
(451313080118390996, 0, 3) Martin Luther King
(451313080118390996, 1, 3) Luther King
(451313080118390996, 2, 3) King
(451313080118390996, 0, 4) Martin Luther King Jr.
(451313080118390996, 1, 4) Luther King Jr.
(451313080118390996, 2, 4) King Jr.
(451313080118390996, 3, 4) Jr.


In [64]:
matcher = Matcher(nlp.vocab)
pattern = [{"POS": "PROPN", "OP": "+"}]
matcher.add("PROPER_NOUN", [pattern], greedy = "LONGEST")
doc = nlp(text)
matches = matcher(doc)
print(len(matches))
for match in matches[:10]:
    print(match, doc[match[1]:match[2]])

59
(451313080118390996, 470, 475) Martin Luther King Jr. Day
(451313080118390996, 537, 542) Martin Luther King Jr. Memorial
(451313080118390996, 0, 4) Martin Luther King Jr.
(451313080118390996, 84, 88) Martin Luther King Sr
(451313080118390996, 129, 133) Southern Christian Leadership Conference
(451313080118390996, 248, 252) Director J. Edgar Hoover
(451313080118390996, 6, 9) Michael King Jr.
(451313080118390996, 326, 329) Nobel Peace Prize
(451313080118390996, 423, 426) James Earl Ray
(451313080118390996, 464, 467) Congressional Gold Medal


In [66]:
matcher = Matcher(nlp.vocab)
pattern = [{"POS": "PROPN", "OP": "+"}, {"POS": "VERB"}]
matcher.add("PROPER_NOUN", [pattern], greedy = "LONGEST")
doc = nlp(text)
matches = matcher(doc)
matches.sort(key = lambda x: x[1])
print(len(matches))
for match in matches[:10]:
    print(match, doc[match[1]:match[2]])

6
(451313080118390996, 50, 52) King advanced
(451313080118390996, 90, 92) King participated
(451313080118390996, 114, 116) King led
(451313080118390996, 248, 253) Director J. Edgar Hoover considered
(451313080118390996, 323, 325) King won
(451313080118390996, 486, 489) United States beginning


In [71]:
import json
with open("data/alice.json", "r") as f:
    data = json.load(f)

In [72]:
text = data[0][2][0]
print(text)

Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, `and what is the use of a book,' thought Alice `without pictures or conversation?'


In [73]:
text = text.replace("`", "'")

In [74]:
print(text)

Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, 'and what is the use of a book,' thought Alice 'without pictures or conversation?'


In [75]:
matcher = Matcher(nlp.vocab)
pattern = [{"ORTH":"'"}, {"IS_ALPHA": True,"OP": "+" }, {"IS_PUNCT":True, "OP": "*"},
          {"ORTH":"'"}]
matcher.add("PROPER_NOUN", [pattern], greedy = "LONGEST")
doc = nlp(text)
matches = matcher(doc)
matches.sort(key = lambda x: x[1])
print(len(matches))
for match in matches[:10]:
    print(match, doc[match[1]:match[2]])

2
(451313080118390996, 47, 58) 'and what is the use of a book,'
(451313080118390996, 60, 67) 'without pictures or conversation?'


In [82]:
speak_lemmas = ["think", "say"]
matcher = Matcher(nlp.vocab)
pattern = [{"ORTH":"'"}, {"IS_ALPHA": True,"OP": "+" }, {"IS_PUNCT":True, "OP": "*"},
          {"ORTH":"'"},
          {"POS": "VERB", "LEMMA": {"IN": speak_lemmas}},
           {"POS":"PROPN", 'OP':"+"},
           {"ORTH":"'"}, {"IS_ALPHA": True,"OP": "+" }, {"IS_PUNCT":True, "OP": "*"},
          {"ORTH":"'"} 
          ]
matcher.add("PROPER_NOUN", [pattern], greedy = "LONGEST")
doc = nlp(text)
matches = matcher(doc)
matches.sort(key = lambda x: x[1])
print(len(matches))
for match in matches[:10]:
    print(match, doc[match[1]:match[2]])

1
(451313080118390996, 47, 67) 'and what is the use of a book,' thought Alice 'without pictures or conversation?'


In [83]:
for text in data[0][2]:
    text = text.replace("`", "'")
    doc = nlp(text)
    matches = matcher(doc)
    matches.sort(key = lambda x: x[1])
    for match in matches[:10]:
        print(match, doc[match[1]:match[2]])

(451313080118390996, 47, 67) 'and what is the use of a book,' thought Alice 'without pictures or conversation?'
